In [54]:
import cv2
import numpy as np
import mediapipe as mp
from math import hypot #hypot calculates the eucledian distance
import screen_brightness_control as sbc #for brightness control
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume #helps us to interact with audio of the system
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL

In [55]:
def main(): #ML PROJECT
    #below four lines give us access to volume of the system
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume)) #volume obj has access to the interface and is uswd below
    volRange = volume.GetVolumeRange()
    minVol, maxVol, _ = volRange
    #setting up the hands model
    mpHands=mp.solutions.hands

    hands = mpHands.Hands(static_image_mode=False, max_num_hands=2, model_complexity = 1, min_detection_confidence = 0.75, min_tracking_confidence = 0.75)
    draw = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)
    try:
        while cap.isOpened():
            ret, frame=cap.read()
            if not ret:
                break
            frame =cv2.flip(frame, 1)
            
            #frame in RGB format
            frameRGB= cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            processed=hands.process(frameRGB)

            left_landmark_list, right_landmark_list = get_left_right_landmarks(frame, processed, draw, mpHands)

            if left_landmark_list:
                #for brightness left hand
                left_distance = get_distance(frame, left_landmark_list)
                #adjusting lefthanddist value to value between 0 to 100 using interpolate
                #interpolate takes in our dist value and converts it into the 0 to 100 range for our brightness
                b_level = np.interp(left_distance, [50, 220], [0,100])
                sbc.set_brightness(b_level)

            
            if right_landmark_list:
                right_distance = get_distance(frame, right_landmark_list)
                #for volume right hand
                vol = np.interp(right_distance, [50, 220], [minVol,maxVol])
                volume.SetMasterVolumeLevel(vol, None)


            cv2.imshow('frame', frame)

            if cv2.waitKey(1) & 0xFF==27:
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

def get_left_right_landmarks(frame, processed, draw, hands):
    left_landmarks_list = []
    right_landmarks_list = []
    
    if processed.multi_hand_landmarks:
        #handlm contains either left or right hand landmarks, idx(index range is 0,21)
        for handlm in processed.multi_hand_landmarks:
            for idx, found_landmark in enumerate(handlm.landmark):
                height, width, _ = frame.shape
                #to get true landmarks ie x, y values is as foll
                x, y = int(found_landmark.x * width), int(found_landmark.y * height)

                if idx== 4 or idx== 8:
                    landmark = [idx, x, y]

                    if handlm == processed.multi_hand_landmarks[0]:
                        left_landmarks_list.append(landmark)
                    elif handlm == processed.multi_hand_landmarks[1]:
                        right_landmarks_list.append(landmark)
                        
            draw.draw_landmarks(frame, handlm, hands.HAND_CONNECTIONS)  #draws all hand connections 
            
    return left_landmarks_list, right_landmarks_list


#to calculate distance between two points (x1,y1)thumb, (x2,y2)index finger
def get_distance(frame, landmark_list):
    if len(landmark_list)<2:
        return 
        
    (x1, y1) , (x2, y2) = (landmark_list[0][1], landmark_list[0][2]), \
         (landmark_list[1][1], landmark_list[1][2])
    cv2.circle(frame, (x1, y1), 7, (0,255,0), cv2.FILLED) #to be able to identify the index and thumb for us visually
    cv2.circle(frame, (x2, y2), 7, (0,255,0), cv2.FILLED)
    cv2.line(frame, (x1, y1), (x2, y2), (0,255,0), 3)

    L = hypot(x2-x1, y2-y1) #returns between the thumb and index finger- green line 
    return L

In [56]:
if __name__ == '__main__' :
    main();

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
[landmark {
  x: 0.740863621
  y: 0.7099666
  z: -4.77914e-007
}
landmark {
  x: 0.688123167
  y: 0.703339517
  z: -0.0145000275
}
landmark {
  x: 0.626085103
  y: 0.659288764
  z: -0.0249768738
}
landmark {
  x: 0.583966792
  y: 0.619952559
  z: -0.0386986881
}
landmark {
  x: 0.56553781
  y: 0.574742377
  z: -0.050121516
}
landmark {
  x: 0.632272065
  y: 0.545132816
  z: -0.00417064689
}
landmark {
  x: 0.584104657
  y: 0.473464817
  z: -0.03883615